In [1]:
from pathlib import Path
import pandas as pd
import numpy as np


In [37]:
# asc 파일을 줄 별로 읽어오기
dir_asc = Path('.').absolute()/'asc'
file_asc = dir_asc/'Logging.asc'
with open(file_asc, "r") as file:
    lines = file.readlines()

lines

['date Fri Sep 13 03:40:25.108 pm 2024\n',
 'base hex  timestamps absolute\n',
 'internal events logged\n',
 '// version 16.2.0\n',
 '// Measurement UUID: b9330f6a-051b-4f4d-a967-2c68124e7f7b\n',
 'Begin TriggerBlock Fri Sep 13 03:40:25.108 pm 2024\n',
 '   0.000000 Start of measurement\n',
 '   0.000471 ETH 1 STAT Link:Link_up Ports:ECU\n',
 '   0.000867 ETH 1 STAT Link:Link_up LinkSpeed:100Mbit/s Physical:IEEE802.3 Duplex:Full MDI:Crossover Connector:RJ45 Ports:Port5\n',
 '   0.001011 ETH 1 STAT Link:Link_down Connector:RJ45 Ports:Port6\n',
 '   0.040818 ETH 1 BUSSTATISTIC HwRxPkts:0 HwTxPkts:0 HwRxError:0 HwTxError:0 HwRxBytes:0 HwTxBytes:0 HwRxNoBuffer:0 HwSQIValue:6 Ports:ECU\n',
 '   0.100574 ETH 1 BUSSTATISTIC HwRxPkts:0 HwTxPkts:0 HwRxError:0 HwTxError:0 HwRxBytes:0 HwTxBytes:0 HwRxNoBuffer:0 HwSQIValue:6 Ports:Port5\n',
 '   0.100695 ETH 1 BUSSTATISTIC HwRxPkts:0 HwTxPkts:0 HwRxError:0 HwTxError:0 HwRxBytes:0 HwTxBytes:0 HwRxNoBuffer:0 HwSQIValue:6 Ports:Port6\n',
 '   0.14081

In [42]:
# 분석 대상 라인들만 추출하기
lines_oi = []
elements_oi = []
for line in lines:
    elements = line.strip().split()

    interesting = True

    # 첫 element가 타임스탬트가 아니면 분석 대상이 아니다. 
    try:
        ts = float(elements[0])
    except ValueError:
        interesting = False

    # Start, STAT, BUSSTATISTIC가 포함된 라인은 분석 대상이 아니다. 
    for element in elements:
        if 'Start' in elements or 'STAT' in element or 'BUSSTATISTIC' in element:
            interesting = False
            break

    # 분석 대상을 lines_oi에 추가한다.
    if interesting:

        # 3번째 element가 Tx인 경우 FCS를 추가한다.
        if elements[3] == 'Tx':
            elements.insert(5, 'FCS:00000000')

        elements_oi.append(elements)
        lines_oi.append(line.strip())

file_new_asc = file_asc.with_suffix('.new.asc')
with open(file_new_asc, "w") as file:
    for line in lines_oi:
        file.write(line + "\n")

elements_oi


[['0.161434',
  'ETH',
  '1',
  'Rx',
  '42:FFFFFFFFFFFF0200000010018100E006080045000030684900004011664EC6122001C612FFFFC351C351001CC3C8FFFFFF010000000CFF01060E0306010001000000',
  'FCS:233b7b26',
  'Ports:Port5',
  'Sim:0'],
 ['0.161434',
  'ETH',
  '1',
  'Tx',
  '42:FFFFFFFFFFFF0200000010018100E006080045000030684900004011664EC6122001C612FFFFC351C351001CC3C8FFFFFF010000000CFF01060E0306010001000000',
  'FCS:00000000',
  'Ports:ECU',
  'Sim:0'],
 ['1.161448',
  'ETH',
  '1',
  'Rx',
  '42:FFFFFFFFFFFF0200000010018100E006080045000030684A00004011664DC6122001C612FFFFC351C351001CC3C7FFFFFF010000000CFF01060F0306010001000000',
  'FCS:eb00c426',
  'Ports:Port5',
  'Sim:0'],
 ['1.161449',
  'ETH',
  '1',
  'Tx',
  '42:FFFFFFFFFFFF0200000010018100E006080045000030684A00004011664DC6122001C612FFFFC351C351001CC3C7FFFFFF010000000CFF01060F0306010001000000',
  'FCS:00000000',
  'Ports:ECU',
  'Sim:0'],
 ['2.161462',
  'ETH',
  '1',
  'Rx',
  '42:FFFFFFFFFFFF0200000010018100E006080045000030684B00004011

In [39]:
# list를 DataFrame으로 변환하기
df = pd.DataFrame(lines_oi, columns=['ts', 'type', 'ch', 'tx_rx', 'data', 'fcs', 'port', 'sim'])
df['ts'] = df['ts'].astype(float)
df['ch'] = df['ch'].astype(int)

df


,ts,type,ch,tx_rx,data,fcs,port,sim
0,0.161434,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:233b7b26,Ports:Port5,Sim:0
1,0.161434,ETH,1,Tx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:00000000,Ports:ECU,Sim:0
2,1.161448,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:eb00c426,Ports:Port5,Sim:0
3,1.161449,ETH,1,Tx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:00000000,Ports:ECU,Sim:0
4,2.161462,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:c0402bfc,Ports:Port5,Sim:0
...,...,...,...,...,...,...,...,...
65,32.161877,ETH,1,Tx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:00000000,Ports:ECU,Sim:0
66,33.161890,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:1263114e,Ports:Port5,Sim:0
67,33.161891,ETH,1,Tx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:00000000,Ports:ECU,Sim:0
68,34.161904,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:3923fe94,Ports:Port5,Sim:0


In [60]:
def extract_lines_oi_from_asc(file_asc: Path):
    """
    Extract lines of interest from an ASC file.
    
    Parameters:
    file_asc (Path): Path to the input ASC file.
    
    Returns:
    list: Lines of interest extracted from the ASC file.
    """
    with open(file_asc, "r") as file:
        lines = file.readlines()

    lines_oi = []
    elements_oi = []
    for line in lines:
        elements = line.strip().split()

        interesting = True

        # 첫 element가 타임스탬트가 아니면 분석 대상이 아니다. 
        try:
            ts = float(elements[0])
        except ValueError:
            interesting = False

        # Start, STAT, BUSSTATISTIC가 포함된 라인은 분석 대상이 아니다. 
        for element in elements:
            if 'Start' in elements or 'STAT' in element or 'BUSSTATISTIC' in element:
                interesting = False
                break

        # 분석 대상을 lines_oi에 추가한다.
        if interesting:

            # 3번째 element가 Tx인 경우 FCS를 추가한다.
            if elements[3] == 'Tx':
                fcs_found = False
                for element in elements[4:]:
                    if 'FCS' in element:
                        fcs_found = True
                        break

                if not fcs_found:
                    elements.insert(5, 'FCS:00000000')

            elements_oi.append(elements)
            lines_oi.append(line.strip())

    return lines_oi, elements_oi

In [64]:
def lines_oi_to_new_asc_and_xlsx(file_asc: Path):
    """
    Convert an ASC file to an XLSX file.
    
    Parameters:
    asc_file (Path): Path to the input ASC file.
    """
    lines_oi, elements_oi = extract_lines_oi_from_asc(file_asc)

    file_new_asc = file_asc.with_suffix('.new.asc')
    with open(file_new_asc, "w") as file:
        for line in lines_oi:
            file.write(line + "\n")

    # list를 DataFrame으로 변환하기
    df = pd.DataFrame(elements_oi, columns=['ts', 'type', 'ch', 'tx_rx', 'data', 'fcs', 'port', 'sim'])
    df['ts'] = df['ts'].astype(float)
    df['ch'] = df['ch'].astype(int)

    # Save DataFrame to XLSX file
    df.to_excel(file_new_asc.with_suffix('.xlsx'), index=False)

    return df

In [65]:
file_asc = Path('.').absolute()/'asc'/'Logging.asc'
df = lines_oi_to_new_asc_and_xlsx(file_asc)

In [66]:
df

,ts,type,ch,tx_rx,data,fcs,port,sim
0,0.161434,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:233b7b26,Ports:Port5,Sim:0
1,0.161434,ETH,1,Tx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:00000000,Ports:ECU,Sim:0
2,1.161448,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:eb00c426,Ports:Port5,Sim:0
3,1.161449,ETH,1,Tx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:00000000,Ports:ECU,Sim:0
4,2.161462,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:c0402bfc,Ports:Port5,Sim:0
...,...,...,...,...,...,...,...,...
65,32.161877,ETH,1,Tx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:00000000,Ports:ECU,Sim:0
66,33.161890,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:1263114e,Ports:Port5,Sim:0
67,33.161891,ETH,1,Tx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:00000000,Ports:ECU,Sim:0
68,34.161904,ETH,1,Rx,42:FFFFFFFFFFFF0200000010018100E00608004500003...,FCS:3923fe94,Ports:Port5,Sim:0


In [67]:
file_asc = Path('.').absolute()/'asc'/'obd_etherrnet_log.asc'
df = lines_oi_to_new_asc_and_xlsx(file_asc)


In [68]:
df

,ts,type,ch,tx_rx,data,fcs,port,sim
0,0.257025,ETH,1,Rx,46:333300000002A81374BD389D86DD6000000000103AF...,FCS:734d6816,Ports:Port6,Sim:0
1,0.257025,ETH,1,Tx,46:333300000002A81374BD389D86DD6000000000103AF...,FCS:00000000,Ports:ECU,Sim:0
2,0.257032,ETH,1,Tx,46:333300000002A81374BD389D86DD6000000000103AF...,FCS:734d6816,Ports:Port5,Sim:0
3,0.272157,ETH,1,Rx,9d:333300010002A81374BD389D86DD60054E170067110...,FCS:40a51d09,Ports:Port6,Sim:0
4,0.272157,ETH,1,Tx,9d:333300010002A81374BD389D86DD60054E170067110...,FCS:00000000,Ports:ECU,Sim:0
...,...,...,...,...,...,...,...,...
807,51.548464,ETH,1,Tx,3c:A81374BD389D02000000100108004500002858E0000...,FCS:f7423f77,Ports:Port6,Sim:0
808,51.550635,ETH,1,Rx,43:A81374BD389D02000000100108004500003558E1000...,FCS:ad1801bb,Ports:Port5,Sim:0
809,51.550636,ETH,1,Tx,43:A81374BD389D02000000100108004500003558E1000...,FCS:ad1801bb,Ports:Port6,Sim:0
810,51.602730,ETH,1,Rx,3c:020000001001A81374BD389D08004500002808C3400...,FCS:e0b51778,Ports:Port6,Sim:0


In [71]:
# df의 data 컬럼을 기준으로 중복되는 행을 제거한다.
df_unique = df.drop_duplicates(subset=['data'], keep='first')
df_unique

,ts,type,ch,tx_rx,data,fcs,port,sim
0,0.257025,ETH,1,Rx,46:333300000002A81374BD389D86DD6000000000103AF...,FCS:734d6816,Ports:Port6,Sim:0
3,0.272157,ETH,1,Rx,9d:333300010002A81374BD389D86DD60054E170067110...,FCS:40a51d09,Ports:Port6,Sim:0
6,0.468434,ETH,1,Rx,52:FFFFFFFFFFFF0200000010010800450000445756000...,FCS:7d8d192,Ports:Port5,Sim:0
9,1.065976,ETH,1,Rx,158:FFFFFFFFFFFFA81374BD389D08004500014A4F3200...,FCS:35b681a9,Ports:Port6,Sim:0
12,1.468449,ETH,1,Rx,52:FFFFFFFFFFFF020000001001080045000044575D000...,FCS:92255390,Ports:Port5,Sim:0
...,...,...,...,...,...,...,...,...
802,51.387607,ETH,1,Rx,3c:020000001001A81374BD389D08004500002808C1400...,FCS:fb685e01,Ports:Port6,Sim:0
804,51.548289,ETH,1,Rx,44:020000001001A81374BD389D08004500003608C2400...,FCS:e8587787,Ports:Port6,Sim:0
806,51.548463,ETH,1,Rx,3c:A81374BD389D02000000100108004500002858E0000...,FCS:f7423f77,Ports:Port5,Sim:0
808,51.550635,ETH,1,Rx,43:A81374BD389D02000000100108004500003558E1000...,FCS:ad1801bb,Ports:Port5,Sim:0


In [72]:
df_unique.to_csv(file_asc.with_suffix('.csv'), index=False)